In [1]:
#!/usr/bin/env python
import os
import itk
import numpy as np

In [2]:
PixelType = itk.ctype('signed short')
Dimension = 3

ImageType = itk.Image[PixelType, Dimension]

[237 203  59]
[306 277 124]

In [8]:
start = itk.Index[Dimension]()
start[0] = 237 # startx
start[1] = 203 # starty
start[2] = 59 # start along Z

In [9]:
end = itk.Index[Dimension]()
end[0] = 306 # endx
end[1] = 277 # endy
end[2] = 124 # size along Z

In [10]:
region = itk.ImageRegion[Dimension]()
region.SetIndex(start)
region.SetUpperIndex(end)

In [11]:
region.GetSize()

itkSize3 ([70, 75, 66])

In [12]:
# Annotation directory
annotations = ['Gastroduodenalis', 'AMS', 'Aorta', 'Pancreas', 'Splenic vein', 'Truncus', 'Vena Cava', 'Vena porta', 'VMI', 'Tumour']
annDir = r'C:\Users\320088652\OneDrive - Philips\TUe\eMTIC\Pancreas_BB\Pancreas_BB\Detail annotaties\Detail Vaat(-)\1051\par\Patient 1051_1051\No study description\Patient 1051 10 S__(10-30-2020_15-49-03-7515)'

In [13]:
dirName = r'C:\Users\320088652\OneDrive - Philips\TUe\eMTIC\Pancreas_BB\Pancreas_BB\CT scans\CT scans Vaat (-)\1051\DICOM\00009A75\AAE6AB16\AAC56CFF\0000E6BD'

In [14]:
namesGenerator = itk.GDCMSeriesFileNames.New()
namesGenerator.SetUseSeriesDetails(True)
namesGenerator.AddSeriesRestriction("0008|0021")
namesGenerator.SetGlobalWarningDisplay(False)
namesGenerator.SetDirectory(dirName)

In [15]:
seriesUID = namesGenerator.GetSeriesUIDs()

In [16]:
seriesUID

('1.2.840.113704.1.111.3040.1457369623.36.101.00512512',)

In [17]:
if len(seriesUID) < 1:
    print('No DICOMs in: ' + dirName)
    sys.exit(1)

In [18]:
print('The directory: ' + dirName)
print('Contains the following DICOM Series: ')
for uid in seriesUID:
    print(uid)



The directory: C:\Users\320088652\OneDrive - Philips\TUe\eMTIC\Pancreas_BB\Pancreas_BB\CT scans\CT scans Vaat (-)\1051\DICOM\00009A75\AAE6AB16\AAC56CFF\0000E6BD
Contains the following DICOM Series: 
1.2.840.113704.1.111.3040.1457369623.36.101.00512512


In [20]:
seriesFound = False
for uid in seriesUID:
    seriesIdentifier = uid

    print('Reading: ' + seriesIdentifier)

    fileNames = namesGenerator.GetFileNames(seriesIdentifier)

    reader = itk.ImageSeriesReader[ImageType].New()
    
    

    dicomIO = itk.GDCMImageIO.New()
    reader.SetImageIO(dicomIO)
    reader.SetFileNames(fileNames)
    reader.ForceOrthogonalDirectionOff()
    header = dicomIO.GetMetaDataDictionary()

    size = itk.size(reader)
    print("Reader")
    print(size)
    print(reader.GetOutput())

    ROI = itk.ExtractImageFilter[ImageType, ImageType].New()
    ROI.SetExtractionRegion(region)
    ROI.SetInput(reader.GetOutput())


    
    spacing = reader.GetOutput().GetSpacing()
    Origin = reader.GetOutput().GetOrigin()
    print("Origin")
    print(Origin)
    Direction = reader.GetOutput().GetDirection()
    print(Direction)



    # IndexToPointMatrix = ROI.GetOutput().GetOrigin()
    # PointToIndexMatrix = ROI.GetOutput().GetOrigin()
    #ROI.Update()

    print("Region of Interest")
    size = itk.size(ROI)
    print(size)
    print(ROI.GetOutput())

   
    #spacing = itk.SpacingType(reader.GetOutput().GetSpacing())
    # header.SetOutputSpacing()
    # header.ChangeSpacingOn()
    # header.UpdateOutputInformation()



    all_labels = []

    for ann in annotations:

        print('ann:', ann)

        labelfilelist = [file for file in os.listdir(annDir) if ann.lower() in file.lower()]

        if len(labelfilelist) > 0:

            labelfile = labelfilelist[0]

            print('labelfile: ', labelfile)
            labelpath = os.path.join(annDir, labelfile)

            
            MeshType = itk.Mesh[itk.SS,3]
            
            meshReader = itk.MeshFileReader[MeshType].New()
            meshReader.SetFileName(labelpath)
            meshReader.Update()

            #print('Meshreaderout: ', meshReader.GetOutput())

            #ImageType = itk.Image[itk.F, 3]

            filter = itk.TriangleMeshToBinaryImageFilter[MeshType, ImageType].New()
            filter.SetInput(meshReader.GetOutput())
            filter.SetInfoImage(ROI.GetOutput())
            filter.Update()

            #print('filterout: ', filter.GetOutput())
            image = np.array(itk.array_from_image(filter.GetOutput())).astype(np.bool)
            all_labels.append(image)

        else:
            
            imsize = reader.GetOutput().GetLargestPossibleRegion().GetSize()

            image = np.zeros((imsize[2], imsize[1], imsize[0])).astype(np.bool)

            all_labels.append(image)

    # process label
    print([ls.shape for ls in all_labels])
    labels = np.stack(all_labels)
    labelmap = labels.astype(np.uint8)
    imsize = ROI.GetOutput().GetLargestPossibleRegion().GetSize()
    #image = np.zeros((imsize[2], imsize[1], imsize[0])).astype(np.bool)
    new_labelmap = np.concatenate((np.zeros((1,imsize[2],imsize[1],imsize[0])).astype(np.uint8), labelmap), axis=0)
    
    labelmap = new_labelmap.argmax(axis=0)
    labelmap = labelmap.astype(np.short)
    #labelmap[:,:,] = 0.
    labelmap[-1, :,:] = 0.
    print(labelmap.shape)

    labelType = ImageType
    labelmap = np.ascontiguousarray(labelmap)
    itk_image = itk.GetImageFromArray(labelmap)
    itk_image.SetMetaDataDictionary(ROI.GetMetaDataDictionary())
    itk_image.Update()

    header = itk.ChangeInformationImageFilter[ImageType].New()
    header.SetInput(itk_image)
    print("Header filter")
    header.SetOutputSpacing(reader.GetOutput().GetSpacing())
    header.ChangeSpacingOn()
    header.SetOutputOrigin(reader.GetOutput().GetOrigin())
    header.ChangeOriginOn()
    header.SetOutputDirection(reader.GetOutput().GetDirection())
    header.ChangeDirectionOn()
    header.UpdateOutputInformation()
    print(header.GetOutput())

    print("itk_image")
    print(itk_image.GetMetaDataDictionary())

    writer = itk.ImageFileWriter[labelType].New()
    writer.SetInput(header.GetOutput())
    segmentation_name = os.path.join(dirName, seriesIdentifier + '_segmentation.nii.gz')
    print(segmentation_name)
    writer.SetFileName(segmentation_name)
    writer.Update()


    metadata = ROI.GetMetaDataDictionary()
    print('metadata: ', metadata)

    data_writer = itk.ImageFileWriter[ImageType].New()
    outFileName = os.path.join(dirName, seriesIdentifier + '_data.nii.gz')
    data_writer.SetFileName(outFileName)
    data_writer.UseCompressionOn()
    data_writer.SetInput(ROI.GetOutput())
    print('Writing: ' + outFileName)
    
    data_writer.Update()

    if seriesFound:
        break

Reading: 1.2.840.113704.1.111.3040.1457369623.36.101.00512512
Reader
itkSize3 ([512, 512, 223])
Image (000001DCC6A919E0)
  RTTI typeinfo:   class itk::Image<short,3>
  Reference Count: 2
  Modified Time: 6018
  Debug: Off
  Object Name: 
  Observers: 
    none
  Source: (000001DCFB016650) 
  Source output name: Primary
  Release Data: Off
  Data Released: False
  Global Release Data: Off
  PipelineMTime: 5966
  UpdateMTime: 0
  RealTimeStamp: 0 seconds 
  LargestPossibleRegion: 
    Dimension: 3
    Index: [0, 0, 0]
    Size: [512, 512, 223]
  BufferedRegion: 
    Dimension: 3
    Index: [0, 0, 0]
    Size: [0, 0, 0]
  RequestedRegion: 
    Dimension: 3
    Index: [0, 0, 0]
    Size: [0, 0, 0]
  Spacing: [0.683594, 0.683594, 1]
  Origin: [-189.386, -27.2289, 245.4]
  Direction: 
1 0 0
0 1 0
0 0 1

  IndexToPointMatrix: 
0.683594 0 0
0 0.683594 0
0 0 1

  PointToIndexMatrix: 
1.46286 0 0
0 1.46286 0
0 0 1

  Inverse Direction: 
1 0 0
0 1 0
0 0 1

  PixelContainer: 
    ImportImageContai